#Homework

Write a python script that will load data from the SpaceX API into DuckDB using dlt.

Use:
- @dlt.source
- @dlt.resource
- @dlt.transformer

SpaceX API URL: https://api.spacexdata.com

Docs: https://github.com/r-spacex/SpaceX-API/blob/master/docs/README.md

Endpoints for loading:
- launches
- rockets
- crew

# Install dlt with duckdb extention

In [ ]:
%%capture
!pip install dlt[duckdb]

# Play with SpaceX API

In [ ]:
import requests
response = requests.get("https://api.spacexdata.com/v4/launches")
response.json()[0]

# Helper
Run the cell and ignore it.

In [ ]:
from dlt.common.pipeline import LoadInfo

def assert_load_info(info: LoadInfo, expected_load_packages: int = 1) -> None:
    """Asserts that expected number of packages was loaded and there are no failed jobs"""
    assert len(info.loads_ids) == expected_load_packages
    # all packages loaded
    assert all(package.state == "loaded" for package in info.load_packages) is True
    # no failed jobs in any of the packages
    info.raise_on_failed_jobs()

# Task 1


Create a pipeline for SpaceX API, for the next endpoints: launches, rockets, crew.

- Fill the empty lines in the functions below.
- `get_rockets` resource should have `table_name=rockets`.
- Create a [resource](https://dlthub.com/docs/general-usage/resource#declare-a-resource) for the `crew` endpoint from scratch.
- [Run the pipeline](https://dlthub.com/docs/walkthroughs/run-a-pipeline) without errors.

In [7]:
import time

import dlt
import requests


@dlt.resource(table_name="launches")
def get_launches():
    # put your code here
    url = "https://api.spacexdata.com/v4/launches"
    response = requests.get(url)
    response.raise_for_status()
    yield response.json()


@dlt.resource(table_name='rockets')
def get_rockets():
    # put your code here
    url = "https://api.spacexdata.com/v4/rockets"
    response = requests.get(url)
    response.raise_for_status()
    yield response.json()


@dlt.resource(table_name='crew')
def get_crew():
    # put your code here
    url = "https://api.spacexdata.com/v4/crew"
    response = requests.get(url)
    response.raise_for_status()
    yield response.json()



pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source',
    destination='duckdb',
    dataset_name='spacex_data',
    dev_mode=True,
)

load_info = pipeline.run([get_launches(), get_rockets(), get_crew()])
print(load_info)
assert_load_info(load_info)


Pipeline spacex_with_source load step completed in 1.29 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_20240905045632
The duckdb destination used duckdb:////content/spacex_with_source.duckdb location to store data
Load package 1725555392.1672015 is LOADED and contains no failed jobs


Run the code below and
## Answer the Question:
- What weight (kg) has the heighest (meters) rocket?

In [8]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM rockets").df()
display(stats_table)

,height__meters,height__feet,diameter__meters,diameter__feet,mass__kg,mass__lb,first_stage__thrust_sea_level__k_n,first_stage__thrust_sea_level__lbf,first_stage__thrust_vacuum__k_n,first_stage__thrust_vacuum__lbf,...,country,company,wikipedia,description,id,_dlt_load_id,_dlt_id,height__feet__v_double,engines__thrust_to_weight__v_double,landing_legs__material
0,22.25,73.0,1.68,5.5,30146,66460,420,94000,480,110000,...,Republic of the Marshall Islands,SpaceX,https://en.wikipedia.org/wiki/Falcon_1,The Falcon 1 was an expendable launch system p...,5e9d0d95eda69955f709d1eb,1725555392.1672015,iR+N6WdM4iqtmw,NaN,NaN,None
1,70.00,NaN,3.70,12.0,549054,1207920,7607,1710000,8227,1849500,...,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_9,Falcon 9 is a two-stage rocket designed and ma...,5e9d0d95eda69973a809d1ec,1725555392.1672015,exno98upqAxN3g,229.6,180.1,carbon fiber
2,70.00,NaN,12.20,39.9,1420788,3125735,22819,5130000,24681,5548500,...,United States,SpaceX,https://en.wikipedia.org/wiki/Falcon_Heavy,With the ability to lift into orbit over 54 me...,5e9d0d95eda69974db09d1ed,1725555392.1672015,3xlXDAz6lzUrRg,229.6,180.1,carbon fiber
3,118.00,387.0,9.00,30.0,1335000,2943000,128000,28775544,138000,31023634,...,United States,SpaceX,https://en.wikipedia.org/wiki/SpaceX_Starship,Starship and Super Heavy Rocket represent a fu...,5e9d0d96eda699382d09d1ee,1725555392.1672015,ZsTXdxqPbdYOeg,NaN,NaN,stainless steel


In [9]:
# What weight (kg) has the heighest (meters) rocket?
result = conn.sql('SELECT mass__kg FROM stats_table ORDER BY height__meters DESC LIMIT 1').df()
print(result)

   mass__kg
0   1335000


# Task 2
- Add pagination, read [SpaceX API doc](https://github.com/r-spacex/SpaceX-API/blob/master/docs/queries.md).
- Combine all resources in one [source](https://dlthub.com/docs/general-usage/source) and the pipeline with `@dlt.source`.
- Add incremental loading for resource `get_launches` using `merge` write disposition, `id` as a  primary key and `dlt.sources.incremental`.
- Run the pipeline [only with](https://dlthub.com/docs/general-usage/source#access-and-select-resources-to-load) `get_launches` resource.

Read more about [incremental loading](https://dlthub.com/docs/general-usage/incremental-loading).

## Try post method to query SpaceX API


In [49]:
import requests

response = requests.post(
    "https://api.spacexdata.com/v4/launches/query",
    json={
        "query": {
            "date_utc": {
                "$gt": "2017-06-22T00:00:00.000Z",
            },
        },
        "options": {
            "page": 1
        }
    }
)
response.json().keys()

dict_keys(['docs', 'totalDocs', 'limit', 'totalPages', 'page', 'pagingCounter', 'hasPrevPage', 'hasNextPage', 'prevPage', 'nextPage'])

Use code above to make your launches resource incremental. Use date_unix 0 as an initial value.

In [52]:
import dlt
import requests

@dlt.resource(
    table_name='launches',
    write_disposition='merge',
    primary_key='id'
)
def get_launches(
    date_unix=dlt.sources.incremental("date_unix", initial_value=0)):
    page = 1
    while True:
        # Get the last value of the incremental cursor
        incremental_value = date_unix.last_value if date_unix.last_value else 0

        response = requests.post(
            "https://api.spacexdata.com/v4/launches/query",
            json={
                "query": {
                    "date_unix": {
                        "$gt": incremental_value
                    }
                },
                "options": {
                    "page": page
                }
            }
        )
        response.raise_for_status()
        data = response.json()

        # Yield the launches data (make sure 'docs' contains the launches)
        yield data["docs"]

        # Handle pagination by checking if there's more data
        if len(data["docs"]) == 0:
            break
        page += 1


@dlt.resource(table_name='rockets')
def get_rockets():
    response = requests.get("https://api.spacexdata.com/v4/rockets")
    response.raise_for_status()
    yield response.json()


@dlt.resource(table_name='crew')
def get_crew():
    response = requests.get("https://api.spacexdata.com/v4/crew")
    response.raise_for_status()
    yield response.json()


@dlt.source
def spacex_source():
    return [get_launches, get_rockets, get_crew]


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_inc',
    destination='duckdb',
    dataset_name='spacex_data_inc',
    dev_mode=True,
)

# Fetch data
data = spacex_source()

# Run the pipeline
load_info = pipeline.run(data)
print(load_info)
assert_load_info(load_info)

# # Run the pipeline one more time, it should load no data
load_info = pipeline.run(data)
print(load_info)

def assert_load_info(load_info, expected_load_packages=None):
    # Print the load_info to understand its structure
    print(load_info)

    # Check if 'load_packages' exists in the load_info
    if 'load_packages' in load_info[1]:
        if expected_load_packages is not None:
            assert len(load_info[1]['load_packages']) == expected_load_packages, \
                f"Expected {expected_load_packages} load packages, but got {len(load_info[1]['load_packages'])}."
    else:
        print("Key 'load_packages' not found in load_info.")

assert_load_info(load_info, expected_load_packages=0)


Pipeline spacex_with_source_inc load step completed in 2.95 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_inc_20240905104022
The duckdb destination used duckdb:////content/spacex_with_source_inc.duckdb location to store data
Load package 1725576022.9119277 is LOADED and contains no failed jobs
Pipeline spacex_with_source_inc load step completed in 2.95 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_inc_20240905104022
The duckdb destination used duckdb:////content/spacex_with_source_inc.duckdb location to store data
Load package 1725576022.9119277 is LOADED and contains no failed jobs
Key 'load_packages' not found in load_info.
Pipeline spacex_with_source_inc load step completed in 2.44 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_inc_20240905104022
The duckdb destination used duckdb:////content/spacex_with_source_inc.duckdb location to store data
Load package

## Answer the Question:
What rocket was launched between 2022-11-01 and 2022-11-02?


In [53]:
conn = duckdb.connect(f'{pipeline.pipeline_name}.duckdb')
conn.sql("show all tables")

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬───────────┐
│       database       │        schema        │         name         │ … │     column_types     │ temporary │
│       varchar        │       varchar        │       varchar        │   │      varchar[]       │  boolean  │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼───────────┤
│ spacex_with_source…  │ spacex_data_inc_20…  │ _dlt_loads           │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_inc_20…  │ _dlt_pipeline_state  │ … │ [BIGINT, BIGINT, V…  │ false     │
│ spacex_with_source…  │ spacex_data_inc_20…  │ _dlt_version         │ … │ [BIGINT, BIGINT, T…  │ false     │
│ spacex_with_source…  │ spacex_data_inc_20…  │ crew                 │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_inc_20…  │ crew__launches       │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_w

In [55]:
spacex_table = conn.sql('SELECT * FROM spacex_data_inc_20240905090200.launches').df()
display(spacex_table)
stats_table = conn.sql("SELECT * FROM spacex_table WHERE date_utc BETWEEN '2022-11-01 00:00:00+00:00' AND '2022-11-02 00:00:00+00:00'").df()
display(stats_table)

,fairings__reused,fairings__recovery_attempt,fairings__recovered,links__patch__small,links__patch__large,links__webcast,links__youtube_id,links__article,links__wikipedia,static_fire_date_utc,...,tbd,id,_dlt_load_id,_dlt_id,links__presskit,links__reddit__launch,links__reddit__media,links__reddit__recovery,links__reddit__campaign,launch_library_id
0,False,False,False,https://images2.imgbox.com/94/f2/NN6Ph45r_o.png,https://images2.imgbox.com/5b/02/QcxHUb5V_o.png,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,2006-03-17 00:00:00+00:00,...,False,5eb87cd9ffd86e000604b32a,1725570120.5367012,QbH6Xt9hG5KWFA,None,None,None,None,None,None
1,False,False,False,https://images2.imgbox.com/f9/4a/ZboXReNb_o.png,https://images2.imgbox.com/80/a2/bkWotCIS_o.png,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaT,...,False,5eb87cdaffd86e000604b32b,1725570120.5367012,cGWTkVSsSqCDpA,None,None,None,None,None,None
2,False,False,False,https://images2.imgbox.com/6c/cb/na1tzhHs_o.png,https://images2.imgbox.com/4a/80/k1oAkY0k_o.png,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaT,...,False,5eb87cdbffd86e000604b32c,1725570120.5367012,7B/ObidvuHCsKA,None,None,None,None,None,None
3,False,False,False,https://images2.imgbox.com/95/39/sRqN7rsv_o.png,https://images2.imgbox.com/a3/99/qswRYzE8_o.png,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,2008-09-20 00:00:00+00:00,...,False,5eb87cdbffd86e000604b32d,1725570120.5367012,yY0GdB8nh6QjZQ,None,None,None,None,None,None
4,False,False,False,https://images2.imgbox.com/ab/5a/Pequxd5d_o.png,https://images2.imgbox.com/92/e4/7Cf6MLY0_o.png,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaT,...,False,5eb87cdcffd86e000604b32e,1725570120.5367012,gGbLHf8mfCVKJA,http://www.spacex.com/press/2012/12/19/spacexs...,None,None,None,None,None
5,NaN,NaN,NaN,https://images2.imgbox.com/73/7f/u7BKqv2C_o.png,https://images2.imgbox.com/66/b4/8KZsjbt4_o.png,https://www.youtube.com/watch?v=nxSxgBKlYws,nxSxgBKlYws,http://www.spacex.com/news/2013/02/12/falcon-9...,https://en.wikipedia.org/wiki/Dragon_Spacecraf...,2010-03-13 00:00:00+00:00,...,False,5eb87cddffd86e000604b32f,1725570120.5367012,g0cTVL9ukSp98w,http://forum.nasaspaceflight.com/index.php?act...,None,None,None,None,None
6,NaN,NaN,NaN,https://images2.imgbox.com/fa/dc/FOUDQ0Sn_o.png,https://images2.imgbox.com/04/6e/kniggvWD_o.png,https://www.youtube.com/watch?v=cdLITgWKe_0,cdLITgWKe_0,https://en.wikipedia.org/wiki/SpaceX_COTS_Demo...,https://en.wikipedia.org/wiki/SpaceX_COTS_Demo...,2010-12-04 00:00:00+00:00,...,False,5eb87cdeffd86e000604b330,1725570120.5367012,epTEJX5IevMZeA,http://www.spacex.com/files/downloads/cots1-20...,None,None,None,None,None
7,NaN,NaN,NaN,https://images2.imgbox.com/c5/f4/XfLVgbaO_o.png,https://images2.imgbox.com/94/8d/YnZ1SLsT_o.png,https://www.youtube.com/watch?v=tpQzDbAY7yI,tpQzDbAY7yI,https://en.wikipedia.org/wiki/Dragon_C2%2B,https://en.wikipedia.org/wiki/Dragon_C2%2B,2012-04-30 00:00:00+00:00,...,False,5eb87cdfffd86e000604b331,1725570120.5367012,UdF7F5wfoigYxQ,https://www.nasa.gov/pdf/649910main_cots2_pres...,None,None,None,None,None
8,NaN,NaN,NaN,https://images2.imgbox.com/3e/91/hlGiK49a_o.png,https://images2.imgbox.com/fb/42/0V9JgYQS_o.png,https://www.youtube.com/watch?v=-Vk3hiV_zXU,-Vk3hiV_zXU,https://www.nasa.gov/mission_pages/station/mai...,https://en.wikipedia.org/wiki/SpaceX_CRS-1,2012-09-29 00:00:00+00:00,...,False,5eb87ce0ffd86e000604b332,1725570120.5367012,7zDtwRiJrdYX1w,https://www.nasa.gov/pdf/694166main_SpaceXCRS-...,None,None,None,None,None
9,NaN,NaN,NaN,https://imag

,fairings__reused,fairings__recovery_attempt,fairings__recovered,links__patch__small,links__patch__large,links__webcast,links__youtube_id,links__article,links__wikipedia,static_fire_date_utc,...,tbd,id,_dlt_load_id,_dlt_id,links__presskit,links__reddit__launch,links__reddit__media,links__reddit__recovery,links__reddit__campaign,launch_library_id


In [54]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
stats_table = conn.sql("SELECT * FROM launches WHERE date_utc BETWEEN '2022-11-01 00:00:00+00:00' AND '2022-11-02 00:00:00+00:00'").df()
display(stats_table)

,fairings__reused,fairings__recovery_attempt,fairings__recovered,links__patch__small,links__patch__large,links__webcast,links__youtube_id,links__article,links__wikipedia,static_fire_date_utc,...,tbd,id,_dlt_load_id,_dlt_id,links__presskit,links__reddit__launch,links__reddit__media,links__reddit__recovery,links__reddit__campaign,launch_library_id
0,NaN,NaN,NaN,None,None,https://youtu.be/pY628jRd6gM,pY628jRd6gM,None,None,NaT,...,False,6243aec2af52800c6e91925d,1725576028.8765225,4eamieHPJFmWrg,None,None,None,None,None,2306e0bc-e1a3-4a4a-9285-e1a94073655e
1,NaN,NaN,NaN,None,None,None,None,None,None,NaT,...,False,6243ba08af52800c6e919270,1725576028.8765225,J+/BVrwRNN3IyA,None,None,None,None,None,None


# Task 3: Get payloads of launches

Use `@dlt.transformer` to get additional info for your data.

Read more about dlt [transformers](https://dlthub.com/docs/general-usage/resource#process-resources-with-dlttransformer).

In [38]:
from os import name
import dlt
import requests

@dlt.resource(table_name='launches')
def get_launches():
    # put your code here
    url = "https://api.spacexdata.com/v4/launches"
    response = requests.get(url)
    response.raise_for_status()
    yield response.json()

@dlt.transformer(data_from=get_launches, table_name='payloads')
def get_payloads(items):
    # put your code here
    for item in items:
      for payload_id in item.get('payloads', []):
        url = f"https://api.spacexdata.com/v4/payloads/{payload_id}"
        response = requests.get(url)
        response.raise_for_status()
        yield response.json()


pipeline = dlt.pipeline(
    pipeline_name='spacex_with_source_tr',
    destination='duckdb',
    dataset_name='spacex_data_tr',
    dev_mode=True,
)

data = get_launches | get_payloads

load_info = pipeline.run(data())
print(load_info)

def assert_load_info(load_info, expected_load_packages=None):
    # Assuming the first element of the tuple is the success flag
    assert load_info[0], "Pipeline run failed!"
    if expected_load_packages is not None:
        # Assuming the second element contains load package information
        assert len(load_info[1]['load_packages']) == expected_load_packages, \
            f"Expected {expected_load_packages} load packages, but got {len(load_info[1]['load_packages'])}."

assert_load_info(load_info)


Pipeline spacex_with_source_tr load step completed in 1.28 seconds
1 load package(s) were loaded to destination duckdb and into dataset spacex_data_tr_20240905101512
The duckdb destination used duckdb:////content/spacex_with_source_tr.duckdb location to store data
Load package 1725574512.1777802 is LOADED and contains no failed jobs


In [39]:
import duckdb
from google.colab import data_table
data_table.enable_dataframe_formatter()

# a database '<pipeline_name>.duckdb' was created in working directory so just connect to it
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
# list all tables
display(conn.sql("DESCRIBE"))

┌──────────────────────┬──────────────────────┬──────────────────────┬───┬──────────────────────┬───────────┐
│       database       │        schema        │         name         │ … │     column_types     │ temporary │
│       varchar        │       varchar        │       varchar        │   │      varchar[]       │  boolean  │
├──────────────────────┼──────────────────────┼──────────────────────┼───┼──────────────────────┼───────────┤
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_loads           │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_pipeline_state  │ … │ [BIGINT, BIGINT, V…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ _dlt_version         │ … │ [BIGINT, BIGINT, T…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ payloads             │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_with_source…  │ spacex_data_tr_202…  │ payloads__customers  │ … │ [VARCHAR, VARCHAR,…  │ false     │
│ spacex_w

Run the code below and

## Answer the Question:

What regime has Satellite "FalconSAT-2" with launch id: 5eb87cd9ffd86e000604b32a?  

In [41]:
stats_table = conn.sql("SELECT * FROM spacex_data_tr_20240905101512.payloads").df()
display(stats_table)

,name,type,reused,launch,mass_kg,mass_lbs,orbit,reference_system,regime,periapsis_km,...,dragon__mass_returned_lbs,dragon__manifest,lifespan_years,longitude,longitude__v_double,mass_lbs__v_double,dragon__mass_returned_kg__v_double,mass_kg__v_double,dragon__mass_returned_lbs__v_double,lifespan_years__v_double
0,FalconSAT-2,Satellite,False,5eb87cd9ffd86e000604b32a,20.0,43.0,LEO,geocentric,low-earth,400.0,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DemoSAT,Satellite,False,5eb87cdaffd86e000604b32b,NaN,NaN,LEO,geocentric,low-earth,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Trailblazer,Satellite,False,5eb87cdbffd86e000604b32c,NaN,NaN,LEO,geocentric,low-earth,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PRESat,Satellite,False,5eb87cdbffd86e000604b32c,NaN,NaN,LEO,geocentric,low-earth,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RatSat,Satellite,False,5eb87cdbffd86e000604b32d,165.0,363.0,LEO,geocentric,low-earth,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Starlink 4-36 (v1.5),Satellite,False,62f3b5200f55c50e192a4e6c,13260.0,29233.0,VLEO,geocentric,very-low-earth,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,"O3b mPower 1,2",Satellite,False,6243ba08af52800c6e919270,5100.0,NaN,MEO,geocentric,medium-earth,NaN,...,NaN,None,NaN,NaN,NaN,11243.58,NaN,NaN,NaN,NaN
219,TTL-1,Satellite,False,62f3b5200f55c50e192a4e6c,NaN,NaN,PO,geocentric,polar,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
220,WorldView Legion 1 & 2,Satellite,False,6243ae58af52800c6e91925a,NaN,NaN,SSO,geocentric,sun-synchronous,NaN,...,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


What regime has Satellite "FalconSAT-2" with launch id: 5eb87cd9ffd86e000604b32a?

In [48]:
result = conn.sql("""
    SELECT regime
    FROM stats_table
    WHERE launch = '5eb87cd9ffd86e000604b32a'
    LIMIT 1
""").df()
print(result)


      regime
0  low-earth
